# Bezier Curves

Just to mess around I want to try and implement Bezier curves.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML

In [ ]:
def interpolate(p0, p1, t):
    """
    Returns a lineraly interpolated point on the line defined by points p0 and p1.
    The line l(t) is parameterized such that l(t=0) = p0 and l(t=1) = p1.
    
    Parameters
    ==========
    po : Point - The initial point of the line.
    p1 : Point - The final point of the line.
    t  : float        - Parameterization variable
    """
    x0, y0 = p0.pos
    x1, y1 = p1.pos
    
    x = (x1 - x0) * t + x0
    y = (y1 - y0) * t + y0
    
    return (x, y)

def points_to_lines(points):
    """
    Takes a list of points and converts it into an x and y array, ready for plotting.
    
    Parameters
    ==========
    points : list of points [(x,y)] - The list of points to convert to lines such that
                                      points[i], points[i+1] each define a new line
    """
    if(isinstance(points[0], tuple)):
        return list(map(lambda p: p[0], points)), list(map(lambda p: p[1], points))
    else:
        return list(map(lambda p: p.x(), points)), list(map(lambda p: p.y(), points))

In [ ]:
class Point():
    def __init__(self, pos, line_parent=None):
        self.pos = pos
        self.line_parent = line_parent
        
    def display(self, ax, color='gray', size=3, alpha=0.5):
        self.marker, = ax.plot(*self.pos, 'o', color=color, markersize=size)
        
    def update(self, pos):
        self.pos = pos
        self.marker.set_data(pos)
        
    def x(self):
        return self.pos[0]
    
    def y(self):
        return self.pos[1]
        
    def __str__(self):
        return str(self.pos)
    
    def __repr__(self):
        return str(self.pos)

class Line():
    def __init__(self, p0, p1):
        self.p0 = p0
        self.p1 = p1
    
    def display(self, ax, color='gray', lw=1):
        x_pts = [self.p0.x(), self.p1.x()]
        y_pts = [self.p0.y(), self.p1.y()]

        self.line, = ax.plot(x_pts, y_pts, color=color, lw=lw, alpha=0.5)
        
    def update(self):

        self.line.set_data(points_to_lines([self.p0, self.p1]))
        
    def __str__(self):
        return f'{self.p0} --> {self.p1}'
    
    def __repr__(self):
        return f'{self.p0} --> {self.p1}'
        
        

In [ ]:
class BezierCurve():
    def __init__(self, points=[]):
        self.points = []
        self.lines = []
        self.final_path_data = []
        
        for p in points:
            self.add_point(p)
            
    def add_point(self, p, level=0):
        if(level >= len(self.points)): 
            self.points.append([])
            
        self.points[level].append(Point(p))
        
        if(len(self.points[level]) > 1):
            self.add_line(self.points[level][-2], self.points[level][-1], level=level)
                    
    def add_line(self, p0, p1, level=0):
        if(level >= len(self.lines)):
            self.lines.append([])
            
        self.lines[level].append(Line(p0, p1))
        
        pos = p0.pos
        self.add_point(pos, level=level+1)
    
    def setup_plot(self):
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(1,1,1)
 
        colors = ['gray', 'blue', 'orange', 'green', 'cyan', 'magenta', 'yellow']
        
        for level, points in enumerate(self.points):
            for point in points:
                if(level == len(self.points) - 1): 
                    point.display(self.ax, color='red', size=5, alpha=1)
                    self.final_path_data.append(point.pos)
                else:
                    point.display(self.ax, color=colors[level % (len(self.points) - 2)])
            
        for level, lines in enumerate(self.lines):
            for line in lines:
                line.display(self.ax, color=colors[level % (len(self.points) - 2)])
                
        self.final_path_curve, = self.ax.plot(points_to_lines(self.final_path_data), color='red')

    def next_frame(self, i):
        for level, lines in enumerate(self.lines):
            for index, line in enumerate(lines):
                line.update()
                
                p0 = self.points[level][index]
                p1 = self.points[level][index + 1]
                
                q = self.points[level + 1][index]
                q.update(interpolate(p0, p1, i / (self.num_frames - 1)))
                
                if(level == len(self.lines) - 1):
                    self.final_path_data.append(q.pos)
        
        self.final_path_curve.set_data(points_to_lines(self.final_path_data))
            
    def create_animation(self, num_frames=100):
        self.num_frames = num_frames
        frame_interval = 40

        anim = animation.FuncAnimation(self.fig, 
                               self.next_frame, 
                               init_func=None,
                               frames=num_frames, 
                               interval=frame_interval, 
                               blit=False,
                               repeat=False)
        
        return anim

In [ ]:
%%capture
points = [(0,0), (10,5), (5,-10), (25,0), (40,2)]
curve = BezierCurve(points)
curve.setup_plot()
anim = curve.create_animation(num_frames=100)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
%%capture
points = [(0,0), (10,-10), (25,5), (5,15), (0,0)]
curve = BezierCurve(points)
curve.setup_plot()
anim = curve.create_animation(num_frames=100)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
%%capture
points = [(0,0), (5,10), (10,5), (15,-20), (20,-30), (30,0), (20,40), (15,0)]
curve = BezierCurve(points)
curve.setup_plot()
anim = curve.create_animation(num_frames=100)

In [ ]:
HTML(anim.to_jshtml())